In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import os
from glob import glob
from itertools import chain

from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

import time

# from os import path

%matplotlib inline

print("All libraries are loaded.")

# ETL
I'm going to load the training samples with which I will train the classifier. These example images come from a combination of the GTI vehicle image database, the KITTI vision benchmark suite, and examples extracted from the project video itself.

In [ ]:
path_to_vehicles = "./dataset/vehicles/"
path_to_nonvehicles = "./dataset/non-vehicles/"

"""
This method is for when I want to load all image file paths into memory
"""
def file_list(path):
    return [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.png'))]

"""
This method is for when I want to get a generator of all image file paths 
"""
def file_generator(path):
    return (chain.from_iterable(glob(os.path.join(x[0], '*.png')) for x in os.walk('.')))

vehicles = file_list(path_to_vehicles)
non_vehicles = file_list(path_to_nonvehicles)

print(len(vehicles))
print(type(vehicles[0]))
print(len(non_vehicles))
print(type(non_vehicles[0]))

In [ ]:
# parameters used in the phase of feature extraction
feat_extraction_params = {'color_space': 'RGB',       # Can be RGB, HSV, LUV, HLS, YUV, YCrCb, BGR
                          'orient': 9,                # HOG orientations
                          'pix_per_cell': 8,          # HOG pixels per cell
                          'cell_per_block': 2,        # HOG cells per block
                          'spatial_size': (32, 32),   # Spatial binning dimensions
                          'nbins': 16,            # Number of histogram bins
                          'hog_channel': "ALL",       # Can be 0, 1, 2, or "ALL"
                          'spatial_feat': True,       # Spatial features on or off
                          'hist_feat': True,          # Histogram features on or off
                          'hog_feat': True,           # HOG features on or off
                          'feature_vec' : True ,       # Call ravel() automatically or not,
                          'transform_sqrt': False,     # Turn "gamma" normalization on or off
                          'visualize': False}          # Turn on visualization or off 

In [ ]:
def convert_color(img, conv='YCrCb'):
    if conv == 'YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB':
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if conv == 'LUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
    if conv == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    if conv == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    if conv == 'YUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

In [ ]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True, transform_sqrt = False):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=transform_sqrt, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=transform_sqrt, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [ ]:
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

In [ ]:
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [ ]:
# Consider using multiple threads here to speed this process up
def load_features(image):
    # loading dictionary values
    color_space = feat_extraction_params['color_space']
    
    orient = feat_extraction_params['orient']
    pix_per_cell = feat_extraction_params['pix_per_cell']
    cell_per_block = feat_extraction_params['cell_per_block']
    spatial_size = feat_extraction_params['spatial_size']
    nbins = feat_extraction_params['nbins']
    
    hog_channel = feat_extraction_params['hog_channel']
    
    feature_vec = feat_extraction_params['feature_vec']
    transform_sqrt = feat_extraction_params['transform_sqrt']
    visualize = feat_extraction_params['visualize']
    
    spatial_feat = feat_extraction_params['spatial_feat']
    hist_feat = feat_extraction_params['hist_feat']
    hog_feat = feat_extraction_params['hog_feat']
    
    
    #1) Define an empty list to receive features
    features = []
    
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'BGR':
        img = convert_color(image, conv=color_space)
    else: 
        feature_image = np.copy(img)      
    
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(img, size=spatial_size)
        
        #4) Append features to list
        features.append(spatial_features)
    
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(img, nbins=nbins)
        
        #6) Append features to list
        features.append(hist_features)
    
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(img.shape[2]):
                hog_features.extend(get_hog_features(img[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(img[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(features)

In [ ]:
def feature_extraction(file_list):
    
    # features will be stores in a list
    features = []
    
    for file in file_list:
        # print(file)
        img = cv2.imread(file)
        
        features.append(load_features(img))
    
    return features

In [ ]:
Xfilename = "./variables/X_values.txt"
yfilename = "./variables/y_values.txt"
loaded = False

In [ ]:
if ((os.path.isfile(Xfilename) != True) and (os.path.isfile(yfilename) != True)): 
    # reassigning the vehicles and non_vehicles variable because I no longer need the paths to the images
    # better to free that memory up
    t1 = time.time()
    vehicles = feature_extraction(vehicles)
    t2 = time.time()
    print("Vehicles processing took", (t2-t1), "seconds.")
    t3 = time.time()
    non_vehicles = feature_extraction(non_vehicles)
    t4 = time.time()
    print("Non-Vehicles processing took", (t4-t3), "seconds.")

In [ ]:
print(type(vehicles))
print(len(vehicles))
print(type(non_vehicles))
print(type(vehicles[0]))
print(type(non_vehicles[0]))
print(vehicles[0].shape)
print(non_vehicles[0].shape)

In [ ]:
print(non_vehicles[0][0])
print(type(non_vehicles[0][0]))

"""
def convertToFloat(src):
    dst = []
    for elem in src:
        print(type(elem))
        print(elem.shape)
        x = np.array(list(elem[:]), dtype=np.float64)
        dst.append(x)
    return dst

vehicles = convertToFloat(vehicles)
non_vehicles = convertToFloat(non_vehicles)

print(non_vehicles[0][0])
print(type(non_vehicles[0][0]))
"""

In [ ]:
if ((os.path.isfile(Xfilename) != True) and (os.path.isfile(yfilename) != True)):
    tup = (vehicles, non_vehicles)
    X = np.vstack(tup)
    # X = np.array(X, dtype = np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(vehicles)), np.zeros(len(non_vehicles))))


    print(vehicles[0].shape)
    print(non_vehicles[0].shape)
else:
    X = np.loadtxt(Xfilename, dtype='int', delimiter=' ')
    y = np.loadtxt(yfilename, dtype='int', delimiter=' ')
    loaded = True

In [ ]:
if !loaded:
    np.savetxt(Xfilename, X, delimiter=" ", fmt="%d")
    np.savetxt(yfilename, y, delimiter=" ", fmt="%d")

In [ ]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient, 'orientations', pix_per_cell, 'pixels per cell and', cell_per_block, 'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()